In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch

Load Tokenizer

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


Load Dataset

In [ ]:
dataset = load_dataset("json", data_files="dataset.json")


Preprocess

In [ ]:
def preprocess_function(examples):
    inputs, targets = [], []

    for c, e in zip(examples["context"], examples["email"]):
        if not isinstance(e, dict) or "body" not in e:
            continue
        email_text = e.get("subject", "") + "\n" + e["body"]
        combined = f"### Instruction:\n{c}\n\n### Response:\n{email_text}"

        inputs.append(combined)
        targets.append(email_text)

    if len(inputs) == 0:
        return {"input_ids": [], "attention_mask": [], "labels": []}

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_dataset = dataset["train"].map(preprocess_function, batched=True, remove_columns=["context", "email"])

Load model in 4-bit

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

Attach LoRA Adapters

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./mistral-email-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    remove_unused_columns=False,
    save_strategy="epoch",
    report_to="none"
)

Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("./mistral-email-lora")

print("✅ Training completed and adapter saved!")

In [ ]:
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")
model = PeftModel.from_pretrained(base_model, "./mistral-email-lora")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)




Gradio UI

In [ ]:
pip install gradio


In [ ]:
import gradio as gr

def generate_email(context):
    prompt = f"### Instruction:\n{context}\n\n### Response:\n"
    out = pipe(prompt, max_new_tokens=250)[0]["generated_text"]
    return out.split("### Response:")[-1].strip()

# Custom CSS for purple, orange, and black theme
custom_css = """
#component-0 {
    background: linear-gradient(135deg, #1a0033 0%, #000000 50%, #331a00 100%) !important;
}
.gradio-container {
    font-family: 'Arial', sans-serif !important;
}
#title {
    text-align: center;
    background: linear-gradient(90deg, #a855f7, #fb923c);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 10px;
}
.input-text textarea {
    background-color: #1a0033 !important;
    border: 2px solid #a855f7 !important;
    color: white !important;
    min-height: 150px !important;
}
.output-text textarea {
    background-color: #1a0033 !important;
    border: 2px solid #fb923c !important;
    color: white !important;
    min-height: 400px !important;
}
button {
    background: linear-gradient(90deg, #a855f7, #fb923c) !important;
    border: none !important;
    color: white !important;
    font-weight: bold !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1 id='title'>✨ Context to Email Generator ✨</h1>")
    gr.Markdown("<p style='text-align: center; color: #fb923c;'>Transform your context into professional emails instantly</p>")

    with gr.Row():
        context_input = gr.Textbox(
            label="📝 Context Input",
            placeholder="Enter your context here... (e.g., 'Reply to customer complaint about late delivery')",
            lines=8,
            elem_classes="input-text"
        )

    generate_btn = gr.Button("🚀 Generate Email", size="lg")

    with gr.Row():
        email_output = gr.Textbox(
            label="📧 Generated Email",
            lines=20,
            elem_classes="output-text"
        )

    generate_btn.click(fn=generate_email, inputs=context_input, outputs=email_output)

demo.launch()